In [1]:
%reload_ext jarvis

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\casteran\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\casteran\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\casteran\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     C:\Users\casteran\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!


Log file storage activated
ERROR, WARNING and INFO messages display activated
Open Modelica simulation activated
plantUml.jar is not up-to-date, see latest release https://github.com/plantuml/plantuml/releases/latest
Hello casteran
I am Jarvis. How may I assist you?


In [2]:
%retrieve_pkg_version


ipython==8.27.0
lxml==5.3.0
notebook==7.2.2
plantuml==0.3.0
jarvis4se==1.4.1
pandas==1.4.1
requests==2.32.5
nltk==3.9.1
PyZMQ==26.2.0
OMPython==3.5.1 
python==3.10.1


# Introduction

This example illustrates the coupling between JARVIS4SE and the DEPS language described [here](https://www.documentation.depslink.fr/)

It is based on the second problem described in the paper [DEPS: a model- and property-based language for system synthesis problems](https://www.depslink.fr/wp-content/uploads/2024/09/DEPS_a_model-_and_property-based_language_for_syst.pdf) called "Software task development on heterogenous embedded drone architecture.

# Embedded drone architecture model

## Concept

This section creates the model of the drone architecture according to the following description:


The poblem deals with __Unmanned Aerial Vehicle (UAV)__ also called drone. The degree of autonomy of these systems evolves progressively from the tracking of a trajectory defined by an operator to a complete autonomy.

In the case of trajectory tracking, a key functionality of the UAV is its ability __to detect obstacles and plan a new trajectory__ that will allow it to avoid the UAV while continuing its path to the desired destination.

The __to detect obstacles and plan a new trajectory__ functionality is provided by a __platform (HW) using a heterogeneous multi-core architecture (SHMC processor)__, which allows the UAV to have a high computing power while reducing its energy consumption (compared to a single-core or homogeneous multi-core processor).

The SHMC processor is composed of cores with different computing power but using the same instruction set.

A core can thus be allocated to any software task, without the need for recompilation. The cores can also be shut down when they are not allocated to
any task, without the need to be kept in a sleep mode.

On this type of processor we generally find a set of more powerful cores (Big Core) and a set of less powerful cores (Little Core).
The processor considered for the problem resolution is made of 4 Big Cores and 4 Little Cores.

Following the 3SE method:
* We create the type "High level functional element" as "Functional element" to allow the recursive decomposition of a "Functional element".
* We create the type "High level function" as "Function" to allow the recursive decomposition of a "Function".

We create the types __Hardware Platform__ and __Core__ as "Functional element".

In [3]:
%%jarvis
with sw_uav
"High level functional element" extends "Functional element"
"High level function" extends "Function"
"Hardware platform" extends "Functional element"
"Core" extends "Functional element"

Creating sw_uav.xml !
High level functional element is a type extending Functional element
High level function is a type extending Function
Hardware platform is a type extending Functional element
Core is a type extending Functional element
sw_uav.xml updated


We create the concept of __Unmanned Aerial Vehicle (UAV)__ as a "High level functional element".

We create the concept of __SHMC - Heterogeneous multi-core architecture platform__ as "Hardware Platform".

We create the concept of __BC1 - BigCore1__, __BC2 - BigCore2__, __BC3 - BigCore3__, __BC4 - BigCore4__, __LC1 - LittleCore1__, __LC2 - LittleCore2__, __LC3 - LittleCore3__, __LC4 - LittleCore4__ as "Core"

In [4]:
%%jarvis
with sw_uav
"UAV - Unnamed Aerial Vehicle" is a high level functional element.
"SHMC - Heterogeneous multi-core architecture platform" is a hardware platform.
"BC1 - BigCore1" is a Core.
"BC2 - BigCore2" is a Core.
"BC3 - BigCore3" is a Core.
"BC4 - BigCore4" is a Core.
"LC1 - LittleCore1" is a Core.
"LC2 - LittleCore2" is a Core.
"LC3 - LittleCore3" is a Core.
"LC4 - LittleCore4" is a Core.

sw_uav.xml parsed
UAV - Unnamed Aerial Vehicle is a High level functional element
SHMC - Heterogeneous multi-core architecture platform is a Hardware platform
BC1 - BigCore1 is a Core
BC2 - BigCore2 is a Core
BC3 - BigCore3 is a Core
BC4 - BigCore4 is a Core
LC1 - LittleCore1 is a Core
LC2 - LittleCore2 is a Core
LC3 - LittleCore3 is a Core
LC4 - LittleCore4 is a Core
sw_uav.xml updated


We know that the UAV is made of SHMC.

We know that SHMC is made of 4 Big Cores and 4 Little Cores.

In [5]:
%%jarvis
with sw_uav
UAV is composed of SHMC
SHMC is composed of "BC1 - BigCore1", "BC2 - BigCore2", "BC3 - BigCore3", "BC4 - BigCore4"
SHMC is composed of "LC1 - LittleCore1", "LC2 - LittleCore2", "LC3 - LittleCore3", "LC4 - LittleCore4"

sw_uav.xml parsed
UAV - Unnamed Aerial Vehicle is composed of SHMC - Heterogeneous multi-core architecture platform
SHMC - Heterogeneous multi-core architecture platform is composed of BC1 - BigCore1
SHMC - Heterogeneous multi-core architecture platform is composed of BC2 - BigCore2
SHMC - Heterogeneous multi-core architecture platform is composed of BC3 - BigCore3
SHMC - Heterogeneous multi-core architecture platform is composed of BC4 - BigCore4
SHMC - Heterogeneous multi-core architecture platform is composed of LC1 - LittleCore1
SHMC - Heterogeneous multi-core architecture platform is composed of LC2 - LittleCore2
SHMC - Heterogeneous multi-core architecture platform is composed of LC3 - LittleCore3
SHMC - Heterogeneous multi-core architecture platform is composed of LC4 - LittleCore4
sw_uav.xml updated


The problem consists of deploying ten tasks (image acquisition, filtering, obstacle detection, …) on a maximum of 8 processors (cores) by satisfying a set of constraints. 

To represent these ten tasks:
* we create the concepts of __To capture frame__, __To identify picture__, __To appply Filter 1__, __To appply Filter 2__, __To identify objects__, __To determine region occupancy__, __To predict trajectory__, __To detect obstacle__, __To avoid obstacle__ and __To calculate new trajectory__ as "Function".
* we assume these concepts to participate in the concept __to detect obstacles and plan a new trajectory__ as "High Level Function"

In [6]:
%%jarvis
with sw_uav
"F - To detect obstacles and plan a new trajectory" is an high level function.
"F1 - To capture frame" is a function.
"F2 - To identify picture" is a function.
"F3 - To appply Filter 1" is a function.
"F4 - To appply Filter 2" is a function.
"F5 - To identify objects" is a function.
"F6 - To determine region occupancy" is a function.
"F7 - To predict trajectory" is a function.
"F8 - To detect obstacle" is a function.
"F9 - To avoid obstacle" is a function.
"F10 - To calculate new trajectory" is a function.

F1, F2, F3, F4, F5, F6, F7, F8, F9, F10 composes F

sw_uav.xml parsed
F1 - To capture frame is a Function
F2 - To identify picture is a Function
F3 - To appply Filter 1 is a Function
F4 - To appply Filter 2 is a Function
F5 - To identify objects is a Function
F6 - To determine region occupancy is a Function
F7 - To predict trajectory is a Function
F8 - To detect obstacle is a Function
F9 - To avoid obstacle is a Function
F10 - To calculate new trajectory is a Function
F - To detect obstacles and plan a new trajectory is a High level function
F - To detect obstacles and plan a new trajectory is composed of F1 - To capture frame
F - To detect obstacles and plan a new trajectory is composed of F2 - To identify picture
F - To detect obstacles and plan a new trajectory is composed of F3 - To appply Filter 1
F - To detect obstacles and plan a new trajectory is composed of F4 - To appply Filter 2
F - To detect obstacles and plan a new trajectory is composed of F5 - To identify objects
F - To detect obstacles and plan a new trajectory is compos

The Hardware Platform is performing the High level function

In [7]:
%%jarvis
with sw_uav
UAV allocates F
SHMC allocates F1, F2, F3, F4, F5, F6, F7, F8, F9, F10

sw_uav.xml parsed
Function F - To detect obstacles and plan a new trajectory is allocated to FunctionalElement UAV - Unnamed Aerial Vehicle
Function F7 - To predict trajectory is allocated to FunctionalElement UAV - Unnamed Aerial Vehicle
Function F8 - To detect obstacle is allocated to FunctionalElement UAV - Unnamed Aerial Vehicle
Function F9 - To avoid obstacle is allocated to FunctionalElement UAV - Unnamed Aerial Vehicle
Function F10 - To calculate new trajectory is allocated to FunctionalElement UAV - Unnamed Aerial Vehicle
Function F5 - To identify objects is allocated to FunctionalElement UAV - Unnamed Aerial Vehicle
Function F1 - To capture frame is allocated to FunctionalElement UAV - Unnamed Aerial Vehicle
Function F4 - To appply Filter 2 is allocated to FunctionalElement UAV - Unnamed Aerial Vehicle
Function F2 - To identify picture is allocated to FunctionalElement UAV - Unnamed Aerial Vehicle
Function F6 - To determine region occupancy is allocated to FunctionalElement UA

A task is characterized by:
* A memory size required for its execution (MemorySize).
* A period (Period).
* Two WCET (Worst Case Execution Time) depending on whether the task is performed by a Big Core (WCETBC) or a Little Core (WCETLC)

We define these concepts as "Attribute"

In [8]:
%%jarvis
with sw_uav
"MemorySize (KB)" is an attribute
"Period (ms)" is an attribute
"BcWCET (ms)" is an attribute
"LcWCET (ms)" is an attribute

sw_uav.xml parsed
MemorySize (KB) is an attribute
Period (ms) is an attribute
BcWCET (ms) is an attribute
LcWCET (ms) is an attribute
sw_uav.xml updated


We define the value of these attributes for each "Function"

In [9]:
%%jarvis
with sw_uav
# T0 : "F1 - To capture frame", coreId, 300, 10, 30, 80, bcRate, lcRate
The "MemorySize (KB)" of F1 is 300.
The "BcWCET (ms)" of F1 is 10.
The "LcWCET (ms)" of F1 is 30.
The "Period (ms)" of F1 is 80.

# T1 : "F2 - To identify picture", coreId, 100, 20, 45, 90, bcRate, lcRate
The "MemorySize (KB)" of F2 is 100.
The "BcWCET (ms)" of F2 is 20.
The "LcWCET (ms)" of F2 is 45.
The "Period (ms)" of F2 is 90.

# T2 : "F3 - To appply Filter 1", coreId, 200, 30, 55, 100, bcRate, lcRate
The "MemorySize (KB)" of F3 is 200.
The "BcWCET (ms)" of F3 is 30.
The "LcWCET (ms)" of F3 is 55.
The "Period (ms)" of F3 is 100.

# T3 : "F4 - To appply Filter 2", coreId, 100, 30, 60, 100, bcRate, lcRate
The "MemorySize (KB)" of F4 is 100.
The "BcWCET (ms)" of F4 is 30.
The "LcWCET (ms)" of F4 is 60.
The "Period (ms)" of F4 is 100.

# T4 : "F5 - To identify objects", coreId, 200, 20, 55, 80, bcRate, lcRate
The "MemorySize (KB)" of F5 is 200.
The "BcWCET (ms)" of F5 is 20.
The "LcWCET (ms)" of F5 is 55.
The "Period (ms)" of F5 is 80.

# T5 : "F6 - To determine region occupancy", coreId, 200, 15, 45, 70, bcRate, lcRate
The "MemorySize (KB)" of F6 is 200.
The "BcWCET (ms)" of F6 is 15.
The "LcWCET (ms)" of F6 is 45.
The "Period (ms)" of F6 is 70.

# T6 : "F7 - To predict trajectory", coreId, 200, 30, 45, 90, bcRate, lcRate
The "MemorySize (KB)" of F7 is 200.
The "BcWCET (ms)" of F7 is 30.
The "LcWCET (ms)" of F7 is 45.
The "Period (ms) of F7 is 90.

# T7 : "F8 - To detect obstacle", coreId, 200, 35, 50, 100, bcRate, lcRate
The "MemorySize (KB)" of F8 is 200.
The "BcWCET (ms)" of F8 is 35.
The "LcWCET (ms)" of F8 is 50.
The "Period (ms)" of F8 is 100.

# T8 : "F9 - To avoid obstacle", coreId, 300, 20, 40, 90, bcRate, lcRate
The "MemorySize (KB)" of F9 is 300.
The "BcWCET (ms)" of F9 is 20.
The "LcWCET (ms)" of F9 is 40.
The "Period (ms)" of F9 is 90.

# T9 : "F10 - To calculate new trajectory", coreId, 400, 40, 60, 100, bcRate, lcRate
The "MemorySize (KB)" of F10 is 400.
The "BcWCET (ms)" of F10 is 40.
The "LcWCET (ms)" of F10 is 60.
The "Period (ms)" of F10 is 100.

sw_uav.xml parsed
Attribute MemorySize (KB) for F1 - To capture frame with value 300
Attribute BcWCET (ms) for F1 - To capture frame with value 10
Attribute LcWCET (ms) for F1 - To capture frame with value 30
Attribute Period (ms) for F1 - To capture frame with value 80
Attribute MemorySize (KB) for F2 - To identify picture with value 100
Attribute BcWCET (ms) for F2 - To identify picture with value 20
Attribute LcWCET (ms) for F2 - To identify picture with value 45
Attribute Period (ms) for F2 - To identify picture with value 90
Attribute MemorySize (KB) for F3 - To appply Filter 1 with value 200
Attribute BcWCET (ms) for F3 - To appply Filter 1 with value 30
Attribute LcWCET (ms) for F3 - To appply Filter 1 with value 55
Attribute Period (ms) for F3 - To appply Filter 1 with value 100
Attribute MemorySize (KB) for F4 - To appply Filter 2 with value 100
Attribute BcWCET (ms) for F4 - To appply Filter 2 with value 30
Attribute LcWCET (ms) for F4 - To appply Filter 2 with value 60
Attri

A Core is characterized by:
* unique core number
* memory used
* maximum memory available


A Big Core is characterized by:
* max memory: 2 000 KB
* max power consumption: 3 Watts
* rate: BcWCET * 2 GHz


A Little Core is characterized by:
* max memory: 512 KB
* max power consumption: 1 Watt
* rate: LcWCET * 1.4 GHz

We define these concepts as "Attribute"

In [10]:
%%jarvis
with sw_uav
"CoreNumber" is an attribute
"MemorySizeUsed (KB)" is an attribute
"MemorySizeMax (KB)" is an attribute
"BcRate (%)" is an attribute
"LcRate (%)" is an attribute
"PowerConsumptionMax (W)" is an attribute

sw_uav.xml parsed
CoreNumber is an attribute
MemorySizeUsed (KB) is an attribute
MemorySizeMax (KB) is an attribute
BcRate (%) is an attribute
LcRate (%) is an attribute
PowerConsumptionMax (W) is an attribute
sw_uav.xml updated


We define the value of these attributes for each "Core"

In [11]:
%%jarvis
with sw_uav
The "CoreNumber" of BC1 is 1.
The "MemorySizeUsed (KB)" of BC1 is 0.
The "MemorySizeMax (KB)" of BC1 is 2000.
The "BcRate (%)" of BC1 is 0.
The "PowerConsumptionMax (W)" of BC1 is 3.

The "CoreNumber" of BC2 is 2.
The "MemorySizeUsed (KB)" of BC2 is 0.
The "MemorySizeMax (KB)" of BC2 is 2000.
The "BcRate (%)" of BC2 is 0.
The "PowerConsumptionMax (W)" of BC2 is 3.

The "CoreNumber" of BC3 is 3.
The "MemorySizeUsed (KB)" of BC3 is 0.
The "MemorySizeMax (KB)" of BC3 is 2000.
The "BcRate (%)" of BC3 is 0.
The "PowerConsumptionMax (W)" of BC3 is 3.

The "CoreNumber" of BC4 is 4.
The "MemorySizeUsed (KB)" of BC4 is 0.
The "MemorySizeMax (KB)" of BC4 is 2000.
The "BcRate (%)" of BC4 is 0.
The "PowerConsumptionMax (W)" of BC4 is 3.

The "CoreNumber" of LC1 is 5.
The "MemorySizeUsed (KB)" of LC1 is 0.
The "MemorySizeMax (KB)" of LC1 is 512.
The "LcRate (%)" of LC1 is 0.
The "PowerConsumptionMax (W)" of LC1 is 1.

The "CoreNumber" of LC2 is 6.
The "MemorySizeUsed (KB)" of LC2 is 0.
The "MemorySizeMax (KB)" of LC2 is 512.
The "LcRate (%)" of LC2 is 0.
The "PowerConsumptionMax (W)" of LC2 is 1.
    
The "CoreNumber" of LC3 is 7.
The "MemorySizeUsed (KB)" of LC3 is 0.
The "MemorySizeMax (KB)" of LC3 is 512.
The "LcRate (%)" of LC3 is 0.
The "PowerConsumptionMax (W)" of LC3 is 1.
    
The "CoreNumber" of LC4 is 8.
The "MemorySizeUsed (KB)" of LC4 is 0.
The "MemorySizeMax (KB)" of LC4 is 512.
The "LcRate (%)" of LC4 is 0.
The "PowerConsumptionMax (W)" of LC4 is 1.

sw_uav.xml parsed
Attribute CoreNumber for BC1 - BigCore1 with value 1
Attribute MemorySizeUsed (KB) for BC1 - BigCore1 with value 0
Attribute MemorySizeMax (KB) for BC1 - BigCore1 with value 2000
Attribute BcRate (%) for BC1 - BigCore1 with value 0
Attribute PowerConsumptionMax (W) for BC1 - BigCore1 with value 3
Attribute CoreNumber for BC2 - BigCore2 with value 2
Attribute MemorySizeUsed (KB) for BC2 - BigCore2 with value 0
Attribute MemorySizeMax (KB) for BC2 - BigCore2 with value 2000
Attribute BcRate (%) for BC2 - BigCore2 with value 0
Attribute PowerConsumptionMax (W) for BC2 - BigCore2 with value 3
Attribute CoreNumber for BC3 - BigCore3 with value 3
Attribute MemorySizeUsed (KB) for BC3 - BigCore3 with value 0
Attribute MemorySizeMax (KB) for BC3 - BigCore3 with value 2000
Attribute BcRate (%) for BC3 - BigCore3 with value 0
Attribute PowerConsumptionMax (W) for BC3 - BigCore3 with value 3
Attribute CoreNumber for BC4 - BigCore4 with value 4
Attribute MemorySizeUsed (KB) for B

## Representation

We represent the problem space as a decomposition diagram of the UAV

In [12]:
%%jarvis
with sw_uav
show decomposition UAV

sw_uav.xml parsed
Decomposition Diagram for UAV generated


Overview :


![figure](http://127.0.0.1:8080/plantuml/svg/hPVRRjiu48RlzHH4W6Ap5mLubhZe1csC3G9iKWJRfhaaQ8csjPL4GPQIUeky-vAIh8ETAdPW-CRM33NVFpCSqX_NfIhAAasyzVSu0uLPw_fRhWgLWi-cr9Sg_BuiJ9L5CvEO0jofN6tKmohElT_Yw0-0XI2S8yW2BpHfRZATbU3i_jCty1xSPvbATGG-wI9M2VYcLt6Ow3EWrg1IZr9MZLsgsYuV6pluyE4wNgv0eXzr0XPL5fQnoUnwdUZKlltYucSJca0U84e49CFGNwvlPpRsjIvrOPOwqwPQWxHAolXzQ0eDL16kubA7PML_v8ag5wP8QwRrAWsbN0qNodgXMoU7w-HsdMDLHVIa1k_g-J1dWY7c1vwP_sjNWhChQj6-6g3oF4ysu2fER320w_0BAcLfP6EIYzeaiOrmrQRWuWByz6x2XzbVNy5vkluT_0auz6vrQehDb_W_3Sx_ldGF4OJUdKsfYRPkxidbQ261tdDRGCevznd2iASKBUMZYIDWvhP9jdLRY0RIMUJMCir8Ns8aUunY5n5tY0moY4X0gTyZ1WrYf5r9TXY3ch4n7GZ9NeB4XoIITObanES2CO9wIl9IkL53ILueDtg5ahsLiijbW3WX246VzvIypsMHnfa6XLvQ0632iCfL5cuQODwbjVQIZPVil5vfWyFO_Lrqr2NOHzmNc0U2zEYiGOyZk_VZnGOqXdK3p6hWhKssjfEaEFXrjsB66UE8ii66GWrcgF9wx2piT6rx5TMChK7MXcb0SjXssWD47I0IdD408ZPeL0ItX4bOsK6cGQQVG5aePxV1MbZOqhPEqZh9tkaDiueUqBoq8oV45uJQEJ18BTxfW3okHtG3ZCSTqDhUqAZxCtLYNlaC2tia2ZJ0z1lClD2HXTZBh5-3jjQ3KthWdjf39SDczGCe6CV3C_HcvWRNJLoMYPxPmu_Ka4beHrPIFtGd8h5ipkZEksSlitS5C3TbQL9W5dk-5lmVrrNdxrpyOIVSgXzRSdi83GptQtjrQ2tGSv4-L-dSpayx9BmxywIBcSdMLPgxvDHlUQWhvUai0WwdxHch469111ehn2E5k5M8XzHuMk7OzxGA-H4ARQS97oDg9-L88HefHAr2DAH6qmh7lgTLo8vGY0Lca8k0ZBhqSkPkKvVnqhsNrlBceRr4pUEbuwLJsaQETdUyBinkAtYOCZgXZEmg8v3uH51AnhMZezhHjdPqM8z9VRk-fwrTS4ntke5jh-fyftQehnrgQuUwaapsvSZnnBL3HzIEOfzGUp7bOgoCzCf8gunqm9DJS-HuOcNa26K81m5dt5wsnifmhmotod07F3ajHuudLeOFK_PiFn_jB_jlzty=)

No update for sw_uav.xml


We can ask about a dedicated element, for example BC1

In [13]:
%%jarvis
with sw_uav
What is BC1 ?

sw_uav.xml parsed


,0
type,Core
alias,BC1
derived from object,none
parent,SHMC - Heterogeneous multi-core architecture platform
child list,none
requirement list,nan
inherited requirement list,none
BcRate (%),0
PowerConsumptionMax (W),3
CoreNumber,1


No update for sw_uav.xml


## Requirements

We translate the following requirement into single requirement REQ_1:

Req 1: A task Ti is deployed on one and only one core.

In [14]:
%%jarvis
with sw_uav
A Core shall host only one function

sw_uav.xml parsed
Requirement identified about Core: A Core shall host only one function



Please give a requirement name:  REQ_1


REQ_1 is a requirement
Requirement REQ_1 is satisfied by Type Core
sw_uav.xml updated


Please note that the requirement is not related to the type "Function", because the relationship between two types cannot be created: it depends on the 3SE model.

We split the following requirement into 3 different requirements REQ_2a, REQ_2b and REQ_2c:

Req 2: Some tasks can only be performed by certain cores according to:
* T1 : 1 ou 5
* T2 : 2 ou 6
* T6 : 3 ou 7

In [15]:
%%jarvis
with sw_uav
The function F2 shall run either on the BC1 or on the LC1.
The function F3 shall run either on the BC2 or on the LC2.
THe function F7 shall run either on the BC3 or on the LC3.

sw_uav.xml parsed
Requirement identified about F2 - To identify picture: The function F2 shall run either on the BC1 or on the LC1
[WARNING] Function F2 - To identify picture is not allocated to Functional element BC1 - BigCore1
[WARNING] Function F2 - To identify picture is not allocated to Functional element LC1 - LittleCore1


Please give a requirement name:  REQ_2a


Requirement identified about F3 - To appply Filter 1: The function F3 shall run either on the BC2 or on the LC2
[WARNING] Function F3 - To appply Filter 1 is not allocated to Functional element BC2 - BigCore2
[WARNING] Function F3 - To appply Filter 1 is not allocated to Functional element LC2 - LittleCore2


Please give a requirement name:  REQ_2b


Requirement identified about F7 - To predict trajectory: THe function F7 shall run either on the BC3 or on the LC3
[WARNING] Function F7 - To predict trajectory is not allocated to Functional element BC3 - BigCore3
[WARNING] Function F7 - To predict trajectory is not allocated to Functional element LC3 - LittleCore3


Please give a requirement name:  REQ_2c


REQ_2a is a requirement
REQ_2b is a requirement
REQ_2c is a requirement
Requirement REQ_2a is satisfied by Function F2 - To identify picture
Requirement REQ_2b is satisfied by Function F3 - To appply Filter 1
Requirement REQ_2c is satisfied by Function F7 - To predict trajectory
sw_uav.xml updated


Please note that the warnings will remain until the allocation of the function to the right core.

This allocation is part of the result of our problem.

We split the following requirement into 3 different requirements REQ_3a, REQ_3b and REQ_3c:

Req 3: Tasks T1 and T3 as well as Task T2 and T4 must necessarily be deployed on the same core. 

Similarly, Task T0 and T1 must necessarily be deployed on different cores


In [16]:
%%jarvis
with sw_uav
The function F2 shall run on the same Core as the function F4.
The function F3 shall run on the same Core as the function F5.
The function F1 shall run on a different Core than the function F2.

sw_uav.xml parsed
Requirement identified about F2 - To identify picture: The function F2 shall run on the same Core as the function F4


Please give a requirement name:  REQ_3a


Requirement identified about F3 - To appply Filter 1: The function F3 shall run on the same Core as the function F5


Please give a requirement name:  REQ_3b


Requirement identified about F1 - To capture frame: The function F1 shall run on a different Core than the function F2


Please give a requirement name:  REQ_3c


REQ_3a is a requirement
REQ_3b is a requirement
REQ_3c is a requirement
Requirement REQ_3a is satisfied by Function F2 - To identify picture
Requirement REQ_3a is satisfied by Type Core
Requirement REQ_3a is satisfied by Function F4 - To appply Filter 2
Requirement REQ_3b is satisfied by Function F3 - To appply Filter 1
Requirement REQ_3b is satisfied by Type Core
Requirement REQ_3b is satisfied by Function F5 - To identify objects
Requirement REQ_3c is satisfied by Function F1 - To capture frame
Requirement REQ_3c is satisfied by Type Core
Requirement REQ_3c is satisfied by Function F2 - To identify picture
sw_uav.xml updated


We rework the following requirement to identify the attributes, and split it into 8 different requirements REQ_4a, REQ_4b, REQ_4c, REQ_4d, REQ_4e, REQ_4f, REQ_4g, REQ_4h (one for each core)

Req 4: The amount of RAM memory available in each core to execute the tasks assigned to it must not be exceeded.

In [17]:
%%jarvis
with sw_uav
# Req 4: The amount of RAM memory available in each core to execute the tasks assigned to it must not be exceeded.
The "MemorySizeUsed (KB)" of BC1 shall be less than its "MemorySizeMax (KB)".
The "MemorySizeUsed (KB)" of BC2 shall be less than its "MemorySizeMax (KB)".
The "MemorySizeUsed (KB)" of BC3 shall be less than its "MemorySizeMax (KB)".
The "MemorySizeUsed (KB)" of BC4 shall be less than its "MemorySizeMax (KB)".
The "MemorySizeUsed (KB)" of LC1 shall be less than its "MemorySizeMax (KB)".
The "MemorySizeUsed (KB)" of LC2 shall be less than its "MemorySizeMax (KB)".
The "MemorySizeUsed (KB)" of LC3 shall be less than its "MemorySizeMax (KB)".
The "MemorySizeUsed (KB)" of LC4 shall be less than its "MemorySizeMax (KB)".

sw_uav.xml parsed
Requirement identified about BC1 - BigCore1: The MemorySizeUsed (KB) of BC1 shall be less than its MemorySizeMax (KB)
[WARNING] Value of Attribute MemorySizeMax (KB) for Functional element BC1 - BigCore1 is not defined by the relationship


Please give a requirement name:  REQ_4a


Requirement identified about BC2 - BigCore2: The MemorySizeUsed (KB) of BC2 shall be less than its MemorySizeMax (KB)
[WARNING] Value of Attribute MemorySizeMax (KB) for Functional element BC2 - BigCore2 is not defined by the relationship


Please give a requirement name:  REQ_4b


Requirement identified about BC3 - BigCore3: The MemorySizeUsed (KB) of BC3 shall be less than its MemorySizeMax (KB)
[WARNING] Value of Attribute MemorySizeMax (KB) for Functional element BC3 - BigCore3 is not defined by the relationship


...

Requirement identified about LC3 - LittleCore3: The MemorySizeUsed (KB) of LC3 shall be less than its MemorySizeMax (KB)
[WARNING] Value of Attribute MemorySizeMax (KB) for Functional element LC3 - LittleCore3 is not defined by the relationship


Please give a requirement name:  REQ_4g


Requirement identified about LC4 - LittleCore4: The MemorySizeUsed (KB) of LC4 shall be less than its MemorySizeMax (KB)
[WARNING] Value of Attribute MemorySizeMax (KB) for Functional element LC4 - LittleCore4 is not defined by the relationship


Please give a requirement name:  REQ_4h


REQ_4a is a requirement
REQ_4b is a requirement
REQ_4c is a requirement
REQ_4d is a requirement
REQ_4e is a requirement
REQ_4f is a requirement
REQ_4g is a requirement
REQ_4h is a requirement
Requirement REQ_4a is satisfied by Attribute MemorySizeUsed (KB)
Requirement REQ_4a is satisfied by FunctionalElement BC1 - BigCore1
Requirement REQ_4a is satisfied by Attribute MemorySizeMax (KB)
Requirement REQ_4b is satisfied by Attribute MemorySizeUsed (KB)
Requirement REQ_4b is satisfied by FunctionalElement BC2 - BigCore2
Requirement REQ_4b is satisfied by Attribute MemorySizeMax (KB)
Requirement REQ_4c is satisfied by Attribute MemorySizeUsed (KB)
Requirement REQ_4c is satisfied by FunctionalElement BC3 - BigCore3
Requirement REQ_4c is satisfied by Attribute MemorySizeMax (KB)
Requirement REQ_4d is satisfied by Attribute MemorySizeUsed (KB)
Requirement REQ_4d is satisfied by FunctionalElement BC4 - BigCore4
Requirement REQ_4d is satisfied by Attribute MemorySizeMax (KB)
Requirement REQ_4e i

Please note that the warnings highlight the dependency of the requirements to the value of the attribute MemorySizeMax (KB).

These value must be specified in other requirements (one per core), creating a dependency between all these requirements.

We won't create these requirements here, because we already set the value of MemorySizeMax for each core.

We rework the following requirement to identify the attributes, and split it into 8 different requirements REQ_5a, REQ_5b, REQ_5c, REQ_5d, REQ_5e, REQ_5f, REQ_5g, REQ_5h (one for each core)

Req 5:The utilization rate of each core, which must not exceed 100%.

In [18]:
%%jarvis
with sw_uav
The "BcRate (%)" of BC1 shall be less than 100 %.
The "BcRate (%)" of BC2 shall be less than 100 %.
The "BcRate (%)" of BC3 shall be less than 100 %.
The "BcRate (%)" of BC4 shall be less than 100 %.
The "LcRate (%)" of LC1 shall be less than 100 %.
The "LcRate (%)" of LC2 shall be less than 100 %.
The "LcRate (%)" of LC3 shall be less than 100 %.
The "LcRate (%)" of LC4 shall be less than 100 %.

sw_uav.xml parsed
Requirement identified about BC1 - BigCore1: The BcRate (%) of BC1 shall be less than 100 %


Please give a requirement name:  REQ_5a


Requirement identified about BC2 - BigCore2: The BcRate (%) of BC2 shall be less than 100 %


Please give a requirement name:  REQ_5b


Requirement identified about BC3 - BigCore3: The BcRate (%) of BC3 shall be less than 100 %


...

Requirement identified about LC3 - LittleCore3: The LcRate (%) of LC3 shall be less than 100 %


Please give a requirement name:  REQ_5g


Requirement identified about LC4 - LittleCore4: The LcRate (%) of LC4 shall be less than 100 %


Please give a requirement name:  REQ_5h


REQ_5a is a requirement
REQ_5b is a requirement
REQ_5c is a requirement
REQ_5d is a requirement
REQ_5e is a requirement
REQ_5f is a requirement
REQ_5g is a requirement
REQ_5h is a requirement
Requirement REQ_5a is satisfied by Attribute BcRate (%)
Requirement REQ_5a is satisfied by FunctionalElement BC1 - BigCore1
Requirement REQ_5b is satisfied by Attribute BcRate (%)
Requirement REQ_5b is satisfied by FunctionalElement BC2 - BigCore2
Requirement REQ_5c is satisfied by Attribute BcRate (%)
Requirement REQ_5c is satisfied by FunctionalElement BC3 - BigCore3
Requirement REQ_5d is satisfied by Attribute BcRate (%)
Requirement REQ_5d is satisfied by FunctionalElement BC4 - BigCore4
Requirement REQ_5e is satisfied by Attribute LcRate (%)
Requirement REQ_5e is satisfied by FunctionalElement LC1 - LittleCore1
Requirement REQ_5f is satisfied by Attribute LcRate (%)
Requirement REQ_5f is satisfied by FunctionalElement LC2 - LittleCore2
Requirement REQ_5g is satisfied by Attribute LcRate (%)
Re

We can check the number of allocated requirement to the Big core BC1:

In [19]:
%%jarvis
with sw_uav
What is BC1?

sw_uav.xml parsed


,0
type,Core
alias,BC1
derived from object,none
parent,SHMC - Heterogeneous multi-core architecture platform
child list,none
requirement list,REQ_5aREQ_4a
inherited requirement list,REQ_3cREQ_3aREQ_1REQ_3b
BcRate (%),0
MemorySizeUsed (KB),0
CoreNumber,1


No update for sw_uav.xml


## Coupling with DEPS language

Waiting for DEPS studio to be available...